#inference

In [ ]:
!pip install tensorflow -q
!pip install tensorflow-addons -q
!pip install opencv-python -q
!pip install scikit-learn -q
!pip install keras -q

ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons


In [ ]:
# Importing the libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import keras
from PIL import Image
from keras import layers
from scipy.special import erf # Importing erf function

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set Image Params
patch_height   = 8
patch_width    = 8
img_height     = 128
img_width      = 128
scaling_factor = 2**10
mha_scaling_factor = 2**10

# Path for Image Source and Parameter
#dataset_path   = "/content/drive/MyDrive/dataset_vision_trands/ExeImg_Dataset/Malware/956dde17805ed5de07358ebed7656536_0.jpg"
#dataset_path   = "/content/drive/MyDrive/dataset_vision_trands/ExeImg_Dataset/Benignware/fce1e7e45cc768dbd55669979806374c_0.jpg"  57%
#dataset_path   = "/content/drive/MyDrive/dataset_vision_trands/ExeImg_Dataset/Benignware/fe63a1786918eb5faf5f0d96730a6068_2.jpg"  55%
#dataset_path   = "/content/drive/MyDrive/dataset_vision_trands/ExeImg_Dataset/Benignware/fe3683b683010e2361cc0ffc6aa82155_1.jpg"    60%
#dataset_path   = "/content/drive/MyDrive/dataset_vision_trands/ExeImg_Dataset/Benignware/fc0b78df7af26182394c6f5e15f0b133_0.jpg"    60%
#dataset_path   = "/content/drive/MyDrive/dataset_vision_trands/ExeImg_Dataset/Benignware/f935d1f09f78044f1cdfa91c07016746_1.jpg"   59.9%
dataset_path    ="/content/drive/MyDrive/drone_vit/drone_dataset/no drone/00000463_(5).jpg"  #61.7 %

# Layer Output
lp_infered_path  = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/patch_encoder_3.txt"
ln1_infered_path = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/layer_normalization_9.txt"
mha_infered_path = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/multi_head_attention_3.txt"
add1_infered_path = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/add_6.txt"
ln2_infered_path  =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/layer_normalization_10.txt"
mlp_infered_path  =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/patches_3.txt"
add2_infered_path = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/add_7.txt"
ln3_infered_path  =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/layer_normalization_11.txt"
fc_infered_path  =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/flatten_3.txt"

# lp_scaled_infered_path   = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/patch_encoder_3_s.txt"
# ln1_scaled_infered_path  = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/layer_normalization_9_s.txt"
# mha_scaled_infered_path  = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/multi_head_attention_3_s.txt"
# add1_scaled_infered_path = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/add_6_s.txt"
# ln2_scaled_infered_path  =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/layer_normalization_10_s.txt"
# mlp_scaled_infered_path  =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/patches_3_s.txt"
# add2_scaled_infered_path = "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/add_7_s.txt"
# ln3_scaled_infered_path  =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/layer_normalization_11_s.txt"
# fc_scaled_infered_path   =  "/content/drive/MyDrive/drone_vit/Outputs/layers_vit_model_dlt/flatten_3_s.txt"

# Weights
lp_weight_path = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/lp_weight.txt"
lp_bias_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/lp_bias.txt"
lp_emb_path    = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/lp_pos_emb.txt"
mha_wq0_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_q0.txt"
mha_wk0_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_k0.txt"
mha_wv0_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_v0.txt"
mha_wq1_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_q1.txt"
mha_wk1_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_k1.txt"
mha_wv1_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_v1.txt"
mha_wqkv_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_qkv.txt"
mha_wouput_path   = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mha_weight_last.txt"
mlp_d0_w_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mlp_weight0.txt"
mlp_d0_b_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mlp_bias0.txt"
mlp_d1_w_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mlp_weight1.txt"
mlp_d1_b_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/mlp_bias1.txt"
fc_d0_w_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/fc_weight0.txt"
fc_d0_b_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/fc_bias0.txt"
fc_d1_w_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/fc_weight1.txt"
fc_d1_b_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/fc_bias1.txt"
fc_d2_w_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/fc_weight2.txt"
fc_d2_b_path  = "/content/drive/MyDrive/drone_vit/Outputs/weights_vit_model_dlt/fc_bias2.txt"

# COE
# position_emb_coe_path = "/content/drive/MyDrive/dataset_vision_trands/ViT/Outputs/lp/position_emb_coe_path.coe"
# lp_coe_path = "/content/drive/MyDrive/dataset_vision_trands/ViT/Outputs/lp/lp_coe_weight.coe"
# bias_coe_path ="/content/drive/MyDrive/dataset_vision_trands/ViT/Outputs/lp/lp_coe_bias.coe "
#mha_wqkv_comb_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/mha_wqkv_comb.coe"
#mha_wq0_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/weight_wq0.coe"
#mha_wk0_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/weight_wk0.coe"
#mha_wv0_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/weight_wv0.coe"
#mha_wq1_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/weight_wq1.coe"
#mha_wk1_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/weight_wk1.coe"
#mha_wv1_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/weight_wv1.coe"
#mha_wo_coe_path = "/content/drive/MyDrive/Project/ViT/Outputs/mha/weight_wo.coe"

# # Scaled Output
# ln1_scaled_path = "/content/drive/MyDrive/dataset_vision_trands/ViT/Outputs/ln1/ln1_data_o.txt"


In [ ]:
  # # Augmentaion
  data_augmentation = keras.Sequential([
      layers.Resizing(img_width, img_height)
  ])

In [ ]:
# GELU
def gelu_exact(x):
    return 0.5 * x * (1 + erf(x / np.sqrt(2)))


In [ ]:
# Scaling and rounding
def scaled_int(input_arry):
  float_array = np.array(input_arry)
  input_scaled = np.round(float_array * scaling_factor).astype(int)
  return input_scaled

In [ ]:
# Convert to 2's complement for an array
def twos_complement(num_array, bits):
    hex_array = []
    np_arry = np.array(num_array).flatten()
    for num in np_arry:
        if num < 0:
            num = (1 << bits) + num  # Apply 2's complement
        hex_value = hex(num)[2:].zfill(bits // 4).upper()  # Format to hex and pad
        hex_array.append(hex_value)
    return np.array(hex_array)

In [ ]:
# Patching
def patch(img,img_width,img_height,patch_width,patch_height):
  patches   = []
  patch_cnt = 0
  # Plotter

  for height_itr in range(0, img_height, patch_height):  # Loop over rows
      for width_itr in range(0, img_width, patch_width):  # Loop over columns
          patch = img[height_itr:height_itr+patch_height, width_itr:width_itr+patch_width]
          patches.append(patch)
          patch_cnt = patch_cnt + 1;

  return patches

In [ ]:
# Linear Projection
def lp(data,patches_num,patch_width,patch_height,scaled_output):

  weight  = np.loadtxt(lp_weight_path, delimiter=",").reshape(192,64)
  bias    = np.loadtxt(lp_bias_path, delimiter=",").reshape(1,64)
  pos_emb = np.loadtxt(lp_emb_path, delimiter=",").reshape(256,64)

  weight_scaled  = np.round(weight * scaling_factor).astype(int)
  bias_scaled    = np.round(bias * scaling_factor).astype(int)
  pos_emb_scaled = np.round(pos_emb * scaling_factor).astype(int)

  embed_img   = []
  embed_img_s = []

  # Flattening
  for patch_count in range(0,patches_num):
    flattened_patch = data[patch_count].reshape(1, 192)

    # Weight Multiplication
    weighted_patch    = np.dot(flattened_patch,weight)
    weighted_patch_s  = np.dot(flattened_patch,weight_scaled)
    weighted_patch_ds = weighted_patch_s

    # Bias addition
    biased_patch   = weighted_patch + bias
    biased_patch_s = weighted_patch_ds + bias_scaled

    # Patch Embedding
    embed_patch   = biased_patch + pos_emb[patch_count]
    embed_patch_s = biased_patch_s + pos_emb_scaled[patch_count]

    # Image Generation
    embed_img.append(embed_patch )
    embed_img_s.append(embed_patch_s)

  if(scaled_output == 1):
    return embed_img_s
  else:
    return embed_img

In [ ]:
 # Layer Normalization 1
def ln1(data,patches_num,patch_width,patch_height,scaled_output):

  ln_array = []
  ln_scaled_array = []

  for patch_count in range(0,patches_num):
    # Row Extraction
    row_array = data[patch_count]
    # Mean
    mean      = np.mean(data[patch_count])
    # Standard Deviation
    stdev     = np.std(row_array, ddof=1)

    # Normilization
    normalized_array = (row_array - mean) / stdev

    # Scaled
    normalized_array_scaled = (row_array - int(mean)) * 256 // int(stdev)
    int_arry = (normalized_array_scaled).astype(int)
    ln_scaled_array.append(int_arry)
    ln_array.append(normalized_array)

  if(scaled_output):
    return ln_scaled_array
  else:
    return ln_array

In [ ]:
# Multi Layer Attention
def softmax(x,scaled_output):
    if(scaled_output==1):
      exp_x = np.exp(x/scaling_factor - np.max(x/scaling_factor, axis=-1, keepdims=True))  # Stable softmax
      return (exp_x / np.sum(exp_x, axis=-1, keepdims=True)) * scaling_factor
    else:
       exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Stable softmax
       return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def mha(data,patches_num,patch_width,patch_height,scaled_output):

  # Weights 1
  wq0 = np.loadtxt(mha_wq0_path, delimiter=",").reshape(64,32)
  wk0 = np.loadtxt(mha_wk0_path, delimiter=",").reshape(64,32)
  wv0 = np.loadtxt(mha_wv0_path, delimiter=",").reshape(64,32)

  # Weights 2
  wq1 = np.loadtxt(mha_wq1_path, delimiter=",").reshape(64,32)
  wk1 = np.loadtxt(mha_wk1_path, delimiter=",").reshape(64,32)
  wv1 = np.loadtxt(mha_wv1_path, delimiter=",").reshape(64,32)
  # Weight output
  w0u = np.loadtxt(mha_wouput_path, delimiter=",").reshape(64,64)

  wq0_scaled =  np.round(wq0 * scaling_factor).astype(int)
  wk0_scaled =  np.round(wk0 * scaling_factor).astype(int)
  wv0_scaled =  np.round(wv0 * scaling_factor).astype(int)

  # Weights 2
  wq1_scaled =  np.round(wq1 * scaling_factor).astype(int)
  wk1_scaled =  np.round(wk1 * scaling_factor).astype(int)
  wv1_scaled =  np.round(wv1 * scaling_factor).astype(int)
  # Weight output
  w0u_scaled =  np.round(w0u * scaling_factor).astype(int)


  mha_array = []
  q0_list   = []
  k0_list   = []
  v0_list   = []
  q1_list   = []
  k1_list   = []
  v1_list   = []
  mha_scaled_array  = []
  q0_scaled_list   = []
  k0_scaled_list   = []
  v0_scaled_list   = []
  q1_scaled_list   = []
  k1_scaled_list   = []
  v1_scaled_list   = []
  for patch_count in range(0,patches_num):
    patch = data[patch_count]  # Extract patch
    #print("patch",patch[0][0])
    # Compute Q, K, V for both heads
    q0, k0, v0 = np.dot(patch, wq0), np.dot(patch, wk0), np.dot(patch, wv0)
    q1, k1, v1 = np.dot(patch, wq1), np.dot(patch, wk1), np.dot(patch, wv1)
    #print("q0",q0[0][0])
    q0_list.append(q0)
    k0_list.append(k0)
    v0_list.append(v0)
    q1_list.append(q1)
    k1_list.append(k1)
    v1_list.append(v1)

    q0_scaled, k0_scaled, v0_scaled = np.dot(patch, wq0_scaled).astype(int), np.dot(patch, wk0_scaled).astype(int), np.dot(patch, wv0_scaled).astype(int)
    q1_scaled, k1_scaled, v1_scaled = np.dot(patch, wq1_scaled).astype(int), np.dot(patch, wk1_scaled).astype(int), np.dot(patch, wv1_scaled).astype(int)

    q0_scaled_list.append(q0_scaled)
    k0_scaled_list.append(k0_scaled)
    v0_scaled_list.append(v0_scaled)
    q1_scaled_list.append(q1_scaled)
    k1_scaled_list.append(k1_scaled)
    v1_scaled_list.append(v1_scaled)

  q0_matrix = np.vstack(q0_list)
  k0_matrix = np.vstack(k0_list)
  v0_matrix = np.vstack(v0_list)
  q1_matrix = np.vstack(q1_list)
  k1_matrix = np.vstack(k1_list)
  v1_matrix = np.vstack(v1_list)

  q0_scaled_matrix = np.vstack(q0_scaled_list)
  k0_scaled_matrix = np.vstack(k0_scaled_list)
  v0_scaled_matrix = np.vstack(v0_scaled_list)
  q1_scaled_matrix = np.vstack(q1_scaled_list)
  k1_scaled_matrix = np.vstack(k1_scaled_list)
  v1_scaled_matrix = np.vstack(v1_scaled_list)
  #print("q0_scaled_matrix",q0_scaled_matrix[0][0])
  #print("k0_scaled_matrix",k0_scaled_matrix[0][0])
  #print("v0_scaled_matrix",v0_scaled_matrix[0][0])
  #print("q0_matrix",q0_matrix[0][0])
  #print("k0_matrix",k0_matrix[0][0])
  #print("v0_matrix",v0_matrix[0][0])

  # # Scaled Dot-Product Attention
 # qkv0_mat = np.dot(softmax(np.dot(q0_matrix, k0_matrix.T) / np.sqrt(64),0), v0_matrix)
  #qkv1_mat = np.dot(softmax(np.dot(q1_matrix, k1_matrix.T) / np.sqrt(64)), v1_matrix)
  qkv0_mat = np.dot(softmax(np.dot(q0_matrix, k0_matrix.T) / np.sqrt(32),0), v0_matrix)
  qkv1_mat = np.dot(softmax(np.dot(q1_matrix, k1_matrix.T) / np.sqrt(32),0), v1_matrix)
  #print(qkv0_mat.shape)
  #print("qkv0_mat",qkv0_mat[0][1])
  #print(qkv1_mat.shape)
  #print("qkv1_mat",qkv1_mat[0][1])


  #qkv0_scaled_mat = np.dot((np.dot(q0_scaled_matrix  , k0_scaled_matrix.T ) // np.sqrt(64)), v0_scaled_matrix) / scaling_factor
  #qkv1_scaled_mat = np.dot((np.dot(q1_scaled_matrix , k1_scaled_matrix.T  ) //  np.sqrt(64)), v1_scaled_matrix )/ scaling_factor
  qkv0_scaled_mat = np.dot(softmax(np.dot(q0_scaled_matrix  , k0_scaled_matrix.T ) // (np.sqrt(32)),1), v0_scaled_matrix) / scaling_factor
  qkv1_scaled_mat = np.dot(softmax(np.dot(q1_scaled_matrix , k1_scaled_matrix.T  ) // ( np.sqrt(32)),1), v1_scaled_matrix )/ scaling_factor
  #print( qkv0_scaled_mat.shape)
  #print ( "qkv0_scaled_mat",qkv0_scaled_mat[0][1])
  #print(qkv1_mat.shape)
  #print( "qkv1_scaled_mat",qkv1_scaled_mat[0][1])

  # tuple(int(x) for x in qkv0_scaled_mat)

  # # if(patch_count == 0):
  # #   print(qkv0_mat.shape)
  #qkvo0_mat = np.dot(qkv0_mat,wo0)
  #qkvo1_mat = np.dot(qkv0_mat,wo1)
  qkv_combined = np.hstack((qkv0_mat,qkv1_mat ))
  qkv_scaled_combined = np.hstack((qkv0_scaled_mat,qkv1_scaled_mat ))
  #qkv_combined = np.hstack((qkvo0_mat,qkvo1_mat ))
  # ## Compute output for attenion
  qkv_final = np.dot(qkv_combined, w0u)
  qkv_scaled_final = np.dot(qkv_scaled_combined,   w0u_scaled)
  int_arry = qkv_scaled_final.astype(int)
  mha_array.append(qkv_final)
  mha_scaled_array.append(int_arry)

  # # print(qkv_combined.shape)
  if(scaled_output == 1):
   # print( mha_scaled_array)
    return mha_scaled_array
  else:
    #print(mha_array)
    return mha_array


In [ ]:
def add(input1,input2,rows):
  add_arry = []
  # Perform element-wise addition
  for i in range(0,rows):
    input1_row = input1[i]
    input2_row = input2[i]
    add_res = np.add(input1_row,input2_row)
    add_arry.append(add_res)
    #print(add_1.shape)
  return add_arry

In [ ]:
 # Layer Normalization 1
def ln2(data,patches_num,patch_width,patch_height,scaled_output):

  ln2_array = []
  ln2_scaled_array = []

  for patch_count in range(0,patches_num):
    # Row Extraction
    row_array = data[patch_count]
    # Mean
    mean      = np.mean(data[patch_count])
    # Standard Deviation
    stdev     = np.std(row_array, ddof=1)

    # Normilization
    normalized_array = (row_array - mean) / stdev

    # Scaled
    normalized_array_scaled = (row_array - int(mean)) * 256 // int(stdev)
    int_arry = (normalized_array_scaled).astype(int)
    ln2_scaled_array.append(int_arry)
    ln2_array.append(normalized_array)

  if(scaled_output):
    return ln2_scaled_array
  else:
    return ln2_array

In [ ]:
# MLP
def mlp(data):

  w_dense0 = np.loadtxt(mlp_d0_w_path, delimiter=",").reshape(64,128)
  b_dense0 = np.loadtxt(mlp_d0_b_path, delimiter=",").reshape(1,128)
  w_dense1 = np.loadtxt(mlp_d1_w_path, delimiter=",").reshape(128,64)
  b_dense1 = np.loadtxt(mlp_d1_b_path, delimiter=",").reshape(1,64)

  data_np_arry = np.array(data)
  mlp_array = []

  # Dense0
  dense0 = np.dot(data_np_arry,w_dense0) + b_dense0 #256x128
  dense0_act = gelu_exact(dense0)

  # Dense1
  dense1 = np.dot(dense0_act,w_dense1) + b_dense1
  dense1_act = gelu_exact(dense1)

  mlp_array.append(dense1_act)

  return mlp_array

In [ ]:
# Fully Connected
def fc(data):

    # Flattening
    data_np_arry = np.array(data).reshape(1,16384)
    #print(data_np_arry.shape)

    # These reshape values are now updated to match your [2048, 1024] model
    w_dense0 = np.loadtxt(fc_d0_w_path, delimiter=",").reshape(16384, 2048) # CHANGED
    b_dense0 = np.loadtxt(fc_d0_b_path, delimiter=",").reshape(1, 2048)     # CHANGED
    w_dense1 = np.loadtxt(fc_d1_w_path, delimiter=",").reshape(2048, 1024)   # CHANGED
    b_dense1 = np.loadtxt(fc_d1_b_path, delimiter=",").reshape(1, 1024)     # CHANGED
    w_dense2 = np.loadtxt(fc_d2_w_path, delimiter=",").reshape(1024, 2)     # CHANGED
    b_dense2 = np.loadtxt(fc_d2_b_path, delimiter=",").reshape(1, 2)        # Stays the same

    fc_array = []

    # Dense0
    dense0 = np.dot(data_np_arry,w_dense0) + b_dense0
    #print("dense0",dense0[0][0])
    dense0_act = gelu_exact(dense0)

    # Dense1
    dense1 = np.dot(dense0_act,w_dense1) + b_dense1
    dense1_act = gelu_exact(dense1)

    # Dense2 (Final Output)
    dense2 = np.dot(dense1_act,w_dense2) + b_dense2
    #dense2 = np.dot(dense1_act,w_dense2)
    print("dense2 ",dense2[0])


    fc_array.append(dense2)

    return fc_array

In [ ]:
# Vision Transfomer

# Inference
def vit_infer():
  # Reading Image
  img_data   = Image.open(dataset_path)

  # Augmentation
  augmented = data_augmentation(np.array(img_data))

  # Convert to NumPy array
  aug_img      = np.array(augmented).astype("uint8")

  # Patching
  patches_num = (img_height // patch_height) * (img_width // patch_width)
  patches     = patch(aug_img,img_width,img_height,patch_width,patch_height)

  # Linear Projection
  lp_array        = lp(patches,patches_num,patch_width,patch_height,0)
  lp_array_scaled = lp(patches,patches_num,patch_width,patch_height,1)
  np.savetxt(lp_infered_path, np.array(lp_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(lp_scaled_infered_path, np.array(lp_array_scaled).flatten(),fmt="%.6f", delimiter=",")

  # Layer Normalization
  ln1_array = ln1(lp_array,patches_num,patch_width,patch_height,0)
  ln1_scaled_array = ln1(lp_array_scaled,patches_num,patch_width,patch_height,1)
  # np.savetxt(ln1_infered_path, np.array(ln1_array).flatten(),fmt="%.6f", delimiter=",")
  # np.savetxt(ln1_scaled_infered_path, np.array(ln1_scaled_array).flatten(),fmt="%d", delimiter=",")

  # Mult-Head Projection
 # Mult-Head Projection
  mha_array = mha(ln1_array,patches_num,patch_width,patch_height,0)
  mha_scaled_array = mha(ln1_scaled_array,patches_num,patch_width,patch_height,1)
  import os
  mha_output_dir = "/content/drive/MyDrive/dataset_vision_trands/ViT/Outputs/mha/"
  os.makedirs(mha_output_dir, exist_ok=True)
  np.savetxt(os.path.join(mha_output_dir, "infered_mha.txt"), np.array(mha_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(os.path.join(mha_output_dir, "infered_mha_scaled.txt"), np.array(mha_scaled_array).flatten(),fmt="%d", delimiter=",")

  mha_np_arry = np.array(mha_array).reshape(256,1,64)
  lp_np_arry = np.array(lp_array)

  # Add1
  lp_np_arry         = np.array(lp_array)
  lp_scaled_np_arry  = np.array(lp_array_scaled )
  mha_np_arry        = np.array(mha_array).reshape(256,1,64)
  mha_scaled_np_arry = np.array(mha_scaled_array).reshape(256,1,64)
  add1_array         = add(lp_np_arry,mha_np_arry,256)
  add1_scaled_array  = add(lp_scaled_np_arry,mha_scaled_np_arry,256)

  np.savetxt(add1_infered_path, np.array(add1_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(add1_scaled_infered_path, np.array(add1_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # LN2
  ln2_array = ln1(add1_array,patches_num,patch_width,patch_height,0)
  ln2_scaled_array = ln1(add1_scaled_array,patches_num,patch_width,patch_height,1)
  # np.savetxt(ln2_infered_path, np.array(ln2_array).flatten(),fmt="%.6f", delimiter=",")
  # np.savetxt(ln2_scaled_infered_path, np.array(ln2_scaled_array).flatten(),fmt="%d", delimiter=",")

  # MLP
  mlp_array = mlp(ln2_array)
  mlp_scaled_array = mlp(ln2_scaled_array,)
  np.savetxt(mlp_infered_path, np.array(mlp_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(mlp_scaled_infered_path, np.array(mlp_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # Add2
  mlp_np_arry         = np.array(mlp_array).reshape(256,1,64)
  mlp_scaled_np_arry  = np.array(mlp_scaled_array).reshape(256,1,64)
  add1_np_arry        = np.array(add1_array)
  add1_scaled_np_arry = np.array(add1_scaled_array)
  add2_array          = add(add1_np_arry,mlp_np_arry,256)
  add2_scaled_array   = add(mlp_scaled_np_arry,mha_scaled_np_arry,256)

  np.savetxt(add2_infered_path, np.array(add2_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(add2_scaled_infered_path, np.array(add2_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # Layer Norm3
  ln3_array = ln1(add2_array,patches_num,patch_width,patch_height,0)
  ln3_scaled_array = ln1(add2_scaled_array,patches_num,patch_width,patch_height,1)
  # np.savetxt(ln3_infered_path, np.array(ln3_array).flatten(),fmt="%.6f", delimiter=",")
  # np.savetxt(ln3_scaled_infered_path, np.array(ln3_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # Fully Connected
  fc_out = fc(ln3_array)
  fc_softmax = softmax(fc_out,0)
  print(fc_softmax [0][0])


vit_infer()

dense2  [-11.53631215 -14.74945451]
[0.96132586 0.03867414]


In [ ]:
# Vision Transfomer

# Inference
def vit_infer():
  # Reading Image
  img_data   = Image.open(dataset_path)

  # Augmentation
  augmented = data_augmentation(np.array(img_data))

  # Convert to NumPy array
  aug_img      = np.array(augmented).astype("uint8")

  # Patching
  patches_num = (img_height // patch_height) * (img_width // patch_width)
  patches     = patch(aug_img,img_width,img_height,patch_width,patch_height)

  # Linear Projection
  lp_array        = lp(patches,patches_num,patch_width,patch_height,0)
  lp_array_scaled = lp(patches,patches_num,patch_width,patch_height,1)
  np.savetxt(lp_infered_path, np.array(lp_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(lp_scaled_infered_path, np.array(lp_array_scaled).flatten(),fmt="%.6f", delimiter=",")

  # Layer Normalization
  ln1_array = ln1(lp_array,patches_num,patch_width,patch_height,0)
  ln1_scaled_array = ln1(lp_array_scaled,patches_num,patch_width,patch_height,1)
  np.savetxt(ln1_infered_path, np.array(ln1_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(ln1_scaled_infered_path, np.array(ln1_scaled_array).flatten(),fmt="%d", delimiter=",")

  # Mult-Head Projection
 # Mult-Head Projection
  mha_array = mha(ln1_array,patches_num,patch_width,patch_height,0)
  mha_scaled_array = mha(ln1_scaled_array,patches_num,patch_width,patch_height,1)
  np.savetxt("/content/drive/MyDrive/dataset_vision_trands/ViT/Outputs/mha/infered_mha.txt", np.array(mha_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt("/content/drive/MyDrive/dataset_vision_trands/ViT/Outputs/mha/infered_mha_scaled.txt", np.array(mha_scaled_array).flatten(),fmt="%d", delimiter=",")

  mha_np_arry = np.array(mha_array).reshape(256,1,64)
  lp_np_arry = np.array(lp_array)

  # Add1
  lp_np_arry         = np.array(lp_array)
  lp_scaled_np_arry  = np.array(lp_array_scaled )
  mha_np_arry        = np.array(mha_array).reshape(256,1,64)
  mha_scaled_np_arry = np.array(mha_scaled_array).reshape(256,1,64)
  add1_array         = add(lp_np_arry,mha_np_arry,256)
  add1_scaled_array  = add(lp_scaled_np_arry,mha_scaled_np_arry,256)

  np.savetxt(add1_infered_path, np.array(add1_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(add1_scaled_infered_path, np.array(add1_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # LN2
  ln2_array = ln1(add1_array,patches_num,patch_width,patch_height,0)
  ln2_scaled_array = ln1(add1_scaled_array,patches_num,patch_width,patch_height,1)
  np.savetxt(ln2_infered_path, np.array(ln2_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(ln2_scaled_infered_path, np.array(ln2_scaled_array).flatten(),fmt="%d", delimiter=",")

  # MLP
  mlp_array = mlp(ln2_array)
  mlp_scaled_array = mlp(ln2_scaled_array,)
  np.savetxt(mlp_infered_path, np.array(mlp_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(mlp_scaled_infered_path, np.array(mlp_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # Add2
  mlp_np_arry         = np.array(mlp_array).reshape(256,1,64)
  mlp_scaled_np_arry  = np.array(mlp_scaled_array).reshape(256,1,64)
  add1_np_arry        = np.array(add1_array)
  add1_scaled_np_arry = np.array(add1_scaled_array)
  add2_array          = add(add1_np_arry,mlp_np_arry,256)
  add2_scaled_array   = add(mlp_scaled_np_arry,mha_scaled_np_arry,256)

  np.savetxt(add2_infered_path, np.array(add2_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(add2_scaled_infered_path, np.array(add2_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # Layer Norm3
  ln3_array = ln1(add2_array,patches_num,patch_width,patch_height,0)
  ln3_scaled_array = ln1(add2_scaled_array,patches_num,patch_width,patch_height,1)
  np.savetxt(ln3_infered_path, np.array(ln3_array).flatten(),fmt="%.6f", delimiter=",")
  np.savetxt(ln3_scaled_infered_path, np.array(ln3_scaled_array).flatten(),fmt="%.6f", delimiter=",")

  # Fully Connected
  fc_out = fc(ln3_array)
  fc_softmax = softmax(fc_out,0)
  print(fc_softmax [0][0])


vit_infer()

dense2  [-11.53631215 -14.74945451]
[0.96132586 0.03867414]


In [ ]:
import numpy as np
from scipy.special import erf

def gelu(x):
    """
    Compute the GELU activation function using the error function (erf).

    Args:
        x (numpy array or float): Input value(s)

    Returns:
        numpy array or float: GELU activated value(s)
    """
    return 0.5 * x * (1 + erf(x / np.sqrt(2)))
import numpy as np
from scipy.special import erf

def gelu(x):
    """
    Compute the GELU activation function using the error function (erf).

    Args:
        x (numpy array or float): Input value(s)

    Returns:
        numpy array or float: GELU activated value(s)
    """
    return 0.5 * x * (1 + erf(x / np.sqrt(2)))

# Example usage
x = np.array([0.450000, -0.250000, 0.650000])
print("GELU Output:", gelu(x))

GELU Output: [ 0.30314015 -0.10032342  0.48240003]


In [ ]:
def gelu_approx(x):
    """
    Compute the approximate GELU activation function using tanh.

    Args:
        x (numpy array or float): Input value(s)

    Returns:
        numpy array or float: GELU activated value(s)
    """
    return 0.5 * x * (1 + np.tanh(np.sqrt(2/np.pi) * (x + 0.044715 * x**3)))

# Example usage
import numpy as np
x = np.array([[2.30 ,   0.80 ,   2.20 ,   0.90],
              [5.10 ,   1.80  ,  5.20  ,  1.90]])
print("Approximate GELU Output:", gelu_approx(x))

Approximate GELU Output: [[2.27567297 0.63043169 2.16967853 0.73422849]
 [5.09999988 1.73525953 5.19999994 1.84545123]]


In [ ]:
# Coe File Generator combine 2 16-bit signed and coverts it to 2s complement
# and generate a COE format file for Xilinx BRAM
def coe_gen(input_file,output_file,rows,cols):

  # Load Input
  input = np.loadtxt(input_file, delimiter=",")
  # Scaling
  input_int    = np.array(input * 1024 , dtype=np.int32)
  input_scaled = np.array(input * 1024 ,dtype=np.int32)
  # Type Casing to Int
  input_repacked = input_int
  # print(len(input_int))
  # print(len(input_repacked))
  for i in range(0,cols):
    for j in range(0,rows):
      input_repacked[i*rows + j] = input_scaled[i + j *32]
      if(i == 1 and j == 1):
        print(input_int[i + j *cols],input_scaled[i + j *cols],input[i + j *cols])

  with open(output_file, 'w+') as file:
    # Write some text
    file.write("memory_initialization_radix=16;\n")
    file.write("memory_initialization_vector=\n")

    for i in range(0, len(input_int), 2):
      # Get two consecutive 16-bit values
      value0     = np.int32(input_repacked[i])
      value1     = np.int32(input_repacked[i+1])
      hex_value0 = f"{value0 & 0xFFFF:04X}"
      hex_value1 = f"{value1 & 0xFFFF:04X}"
      combined_str = hex_value1 + hex_value0

      # Write to file
      file.write(combined_str)

      # Special Case for last line
      if(i == len(input_int) - 2):
          file.write(";")

      else:
        file.write(",\n")

coe_gen(lp_weight_path,lp_coe_path,192,64)


#coe_gen(lp_emb_path ,position_emb_coe_path)
#coe_gen(lp_weight_path,lp_coe_path)
# coe_gen(lp_bias_path,bias_coe_path)
# coe_gen(mha_wk0_path,mha_wk0_coe_path,64,32)
# coe_gen(mha_wv0_path,mha_wv0_coe_path,64,32)
# coe_gen(mha_wq1_path,mha_wq1_coe_path,64,32)
# coe_gen(mha_wk1_path,mha_wk1_coe_path,64,32)
# coe_gen(mha_wv1_path,mha_wv1_coe_path,64,32)
# coe_gen(mha_wqkv_path,mha_wqkv_comb_coe_path,64,32)
# coe_gen(mha_wo_path,mha_wo_coe_path,64,32)

TypeError: coe_gen() missing 2 required positional arguments: 'rows' and 'cols'

In [ ]:
# Coe File Generator combine 2 16-bit signed and coverts it to 2s complement
# and generate a COE format file for Xilinx BRAM
def coe_gen_lp(input_file,output_file):

  # Load Input
  input = np.loadtxt(input_file, delimiter=",")
  # Scaling
  input_int    = np.array(input * 1024, dtype=np.int32)
  input_scaled = np.array(input * 1024,dtype=np.int32)

  with open(output_file, 'w+') as file:
    # Write some text
    file.write("memory_initialization_radix=16;\n")
    file.write("memory_initialization_vector=\n")

    for i in range(0, len(input_int), 2):
      # Get two consecutive 16-bit values
      value0     = np.int32(input_int[i])
      value1     = np.int32(input_int[i+1])
      hex_value0 = f"{value0 & 0xFFFF:04X}"
      hex_value1 = f"{value1 & 0xFFFF:04X}"
      combined_str = hex_value1 + hex_value0

      # Write to file
      file.write(combined_str)

      # Special Case for last line
      if(i == len(input_int) - 2):
          file.write(";")

      else:
        file.write(",\n")


coe_gen_lp(lp_emb_path ,position_emb_coe_path)
coe_gen_lp(lp_bias_path,bias_coe_path)

NameError: name 'position_emb_coe_path' is not defined